<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=584cef15d099883ede2cfd0342dd21c1eb54e30261862e85eac59e3152ec3b2e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 09:13:23
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: 0.00 (0.0%)
Current PnL: -23.34 L (-15.3%)
CY Booked + Current PnL: -9.45 L (-6.19%)
-------------------
Total profit:  1.39 L
Total loss:  -24.73 L
-------------------
Total Booked + Current PnL: 17.53 L (13.95%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.75%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.75 L (62.31%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.0,-14.22,16.68,0.09,14419.0,-14330.0,86447.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.57,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.0,-9.33,10.31,0.02,16317.0,-16292.0,158267.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.67,OX40N,NTT,PAINTS
43,ITC,452.00,0.0,-1.40,12.01,10.44,23700.0,-2802.0,197336.0,-41.15,43.0,X-LC,0.83,5.0,-0.12,1.39,4.09,X40,NTT,FMCG
35,ICICIGI,2252.93,0.0,6.57,12.79,20.19,23760.0,11448.0,185768.0,-16.49,50.0,X-MC,7.57,68.0,0.48,1.30,22.67,X40,ATH,INSURANCE
52,MEDANTA,1486.00,0.0,4.41,18.83,24.08,24960.0,5603.0,132553.0,-4.04,48.0,X-SC,4.91,89.0,0.22,0.93,24.90,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.0,-36.17,76.88,12.90,107220.0,-79032.0,139464.0,105.53,56.0,H-SC,14.32,161.0,-0.74,0.98,15.96,OX40N,NTT,FINANCE
1,ABB,7934.00,0.0,-7.22,56.89,45.57,138093.0,-18883.0,242736.0,-40.33,45.0,H-MC,5.73,120.0,-0.14,1.70,3.29,AR,NTT,ELECTRICAL
2,ABBOTINDIA,35195.00,0.0,-0.98,18.60,17.44,27598.0,-1472.0,148375.0,-14.73,50.0,X-MC,4.28,63.0,-0.05,1.04,17.98,X40,ATH,PHARMA
3,ACC,3906.00,0.0,-19.58,104.11,64.15,199235.0,-46581.0,191370.0,-52.82,65.0,X-SC,1.52,82.0,-0.23,1.34,7.35,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-0.0,-26.94,165.02,93.63,121729.0,-27197.0,73766.0,-21.73,30.0,H-SC,5.95,145.0,-0.22,0.52,9.87,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.0,-36.17,76.88,12.90,107220.0,-79032.0,139464.0,105.53,56.0,H-SC,14.32,161.0,-0.74,0.98,15.96,OX40N,NTT,FINANCE
1,ABB,7934.00,0.0,-7.22,56.89,45.57,138093.0,-18883.0,242736.0,-40.33,45.0,H-MC,5.73,120.0,-0.14,1.70,3.29,AR,NTT,ELECTRICAL
2,ABBOTINDIA,35195.00,0.0,-0.98,18.60,17.44,27598.0,-1472.0,148375.0,-14.73,50.0,X-MC,4.28,63.0,-0.05,1.04,17.98,X40,ATH,PHARMA
3,ACC,3906.00,0.0,-19.58,104.11,64.15,199235.0,-46581.0,191370.0,-52.82,65.0,X-SC,1.52,82.0,-0.23,1.34,7.35,XY24,BTT,CEMENT
4,ALKYLAMINE,4546.37,-0.0,-26.94,165.02,93.63,121729.0,-27197.0,73766.0,-21.73,30.0,H-SC,5.95,145.0,-0.22,0.52,9.87,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.0,-0.84,108.48,106.72,132669.0,-1038.0,122298.0,-50.82,32.0,H-SC,3.69,139.0,-0.01,0.86,22.34,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.0,-5.28,119.50,107.91,167118.0,-7796.0,139848.0,-22.98,32.0,M-SC,10.48,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.0,-9.33,10.31,0.02,16317.0,-16292.0,158267.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.67,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.0,-14.22,16.68,0.09,14419.0,-14330.0,86447.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.57,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.0,-22.57,46.56,13.47,97213.0,-60876.0,208791.0,-68.12,27.0,H-SC,1.39,158.0,-0.63,1.47,6.16,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.0,-23.20,30.20,-0.01,29642.0,-29654.0,98151.0,19986.96,63.0,M-SC,1.69,233.0,-1.00,0.69,11.14,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.0,1.42,69.85,72.27,120039.0,2405.0,171853.0,-12.89,57.0,M-LC,4.52,99.0,0.02,1.21,7.68,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.0,1.42,69.85,72.27,120039.0,2405.0,171853.0,-12.89,57.0,M-LC,4.52,99.0,0.02,1.21,7.68,XR,NTT,IT
38,INDIAMART,4810.62,0.0,-0.84,108.48,106.72,132669.0,-1038.0,122298.0,-50.82,32.0,H-SC,3.69,139.0,-0.01,0.86,22.34,AR,ATH,MISC
25,FINCABLES,1641.55,0.0,-5.28,119.50,107.91,167118.0,-7796.0,139848.0,-22.98,32.0,M-SC,10.48,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.0,-2.28,38.80,35.63,79459.0,-4784.0,204790.0,-15.85,33.0,H-MC,4.38,119.0,-0.06,1.44,14.01,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.0,-5.19,91.87,81.92,99996.0,-5953.0,108845.0,-44.42,32.0,M-SC,4.52,236.0,-0.06,0.76,28.86,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.0,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.57,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.0,-9.37,27.14,15.23,49279.0,-18772.0,181575.0,-22.54,24.0,X-MC,6.65,66.0,-0.38,1.27,19.86,X40N,NTT,REALTY
11,BATAINDIA,2096.00,-0.0,-35.13,108.41,35.20,90493.0,-45197.0,83473.0,9.46,25.0,X-SC,12.87,92.0,-0.50,0.59,0.53,X40,NTT,FOOTWEAR
60,RELAXO,1176.00,-0.0,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,27.0,X-SC,3.95,91.0,-0.48,0.53,1.39,X40N,NTT,FOOTWEAR
22,DIXON,18931.72,0.0,-5.79,29.06,21.58,38365.0,-8118.0,132021.0,-52.49,31.0,X-MC,6.83,56.0,-0.21,0.93,16.88,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.0,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.57,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.00,0.0,-1.40,12.01,10.44,23700.0,-2802.0,197336.0,-41.15,43.0,X-LC,0.83,5.0,-0.12,1.39,4.09,X40,NTT,FMCG
3,ACC,3906.00,0.0,-19.58,104.11,64.15,199235.0,-46581.0,191370.0,-52.82,65.0,X-SC,1.52,82.0,-0.23,1.34,7.35,XY24,BTT,CEMENT
36,ICICIPRULI,790.00,0.0,0.69,30.07,30.97,53511.0,1216.0,177954.0,-22.98,46.0,X-MC,1.55,75.0,0.02,1.25,13.33,X40,ATH,INSURANCE
34,HONAUT,58357.33,0.0,-10.94,60.92,43.32,88371.0,-17812.0,145060.0,-23.50,55.0,X-SC,1.85,90.0,-0.20,1.02,11.38,X40N,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.0,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.57,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.0,-13.50,71.61,48.45,124205.0,-27060.0,173446.0,-19.66,31.0,X-MC,6.81,64.0,-0.22,1.22,0.22,X40N,ATH,FINANCE
56,QUESS,424.00,-0.0,-29.13,101.55,42.84,46785.0,-18935.0,46071.0,-53.72,37.0,X-SC,37.42,83.0,-0.40,0.32,0.25,XY24,NTT,MISC
11,BATAINDIA,2096.00,-0.0,-35.13,108.41,35.20,90493.0,-45197.0,83473.0,9.46,25.0,X-SC,12.87,92.0,-0.50,0.59,0.53,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,0.0,-5.07,32.80,26.07,50984.0,-8300.0,155440.0,-30.06,35.0,X-MC,9.02,55.0,-0.16,1.09,0.76,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.0,-29.13,101.55,42.84,46785.0,-18935.0,46071.0,-53.72,37.0,X-SC,37.42,83.0,-0.40,0.32,0.25,XY24,NTT,MISC
60,RELAXO,1176.00,-0.0,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,27.0,X-SC,3.95,91.0,-0.48,0.53,1.39,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.0,-35.13,108.41,35.20,90493.0,-45197.0,83473.0,9.46,25.0,X-SC,12.87,92.0,-0.50,0.59,0.53,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.0,-5.79,29.06,21.58,38365.0,-8118.0,132021.0,-52.49,31.0,X-MC,6.83,56.0,-0.21,0.93,16.88,X40N,ATH,IT
52,MEDANTA,1486.00,0.0,4.41,18.83,24.08,24960.0,5603.0,132553.0,-4.04,48.0,X-SC,4.91,89.0,0.22,0.93,24.90,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.0,-14.65,39.75,19.28,117371.0,-50683.0,295273.0,-25.57,62.0,X-LC,7.52,1.0,-0.43,2.07,9.16,X40,ATH,IT
41,INFY,2275.00,0.0,5.38,46.96,54.87,154838.0,16836.0,329724.0,-17.80,61.0,X-LC,2.99,2.0,0.11,2.31,14.11,X40,BTT,IT
76,TMPV,600.00,-0.0,-15.09,67.46,42.18,158319.0,-41724.0,234686.0,-24.30,22.0,X-LC,0.57,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.0,-9.89,50.56,35.67,143900.0,-31230.0,284612.0,-20.61,40.0,X-LC,5.96,4.0,-0.22,2.00,2.60,X40N,ATH,FMCG
43,ITC,452.00,0.0,-1.40,12.01,10.44,23700.0,-2802.0,197336.0,-41.15,43.0,X-LC,0.83,5.0,-0.12,1.39,4.09,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.0,-13.23,109.38,81.67,89030.0,-12415.0,81395.0,7316.67,50.0,L-SC,18.73,271.0,-0.14,0.57,57.62,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.0,-12.50,59.48,39.54,52649.0,-12644.0,88515.0,-31.56,67.0,M-SC,6.73,220.0,-0.24,0.62,24.81,AR,ATH,AUTO
51,MASFIN,398.61,0.0,-5.21,28.75,22.05,26703.0,-5100.0,92880.0,-18.25,50.0,H-SC,7.34,164.0,-0.19,0.65,35.01,XR,ATH,FINANCE
70,SURYODAY,216.00,0.0,-19.83,53.46,23.03,76750.0,-35506.0,143565.0,55.70,48.0,H-SC,10.60,167.0,-0.46,1.01,42.45,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.0,-9.33,10.31,0.02,16317.0,-16292.0,158267.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.67,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.0,-9.33,10.31,0.02,16317.0,-16292.0,158267.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.67,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.0,0.64,20.18,20.95,51130.0,1602.0,253370.0,-0.35,76.0,X-LC,13.38,31.0,0.03,1.78,36.50,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.0,-38.28,115.30,32.89,54946.0,-29552.0,47655.0,-688.19,59.0,L-MC,5.57,259.0,-0.54,0.33,31.24,XR,NTT,BANKS
0,5PAISA,593.00,0.0,-36.17,76.88,12.90,107220.0,-79032.0,139464.0,105.53,56.0,H-SC,14.32,161.0,-0.74,0.98,15.96,OX40N,NTT,FINANCE
50,LTIM,7201.88,0.0,8.26,21.61,31.66,57588.0,20339.0,266490.0,3.99,62.0,H-LC,12.77,43.0,0.35,1.87,47.86,X200,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,44.66
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.61
MC    29.27
LC    25.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.96
X40      21.21
X40N     11.91
XR        9.61
XY25      9.06
AR        8.94
OX40N     7.69
X200      1.87
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.86
X-LC    19.96
M-SC    11.99
X-SC     8.32
H-MC     4.71
H-LC     2.98
L-SC     1.41
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-5.04,39.01
IT,13.78,-15.26,74.75
FINANCE,9.39,-15.84,64.78
MISC,7.37,-23.76,74.47
PAINTS,6.01,-7.99,24.03
ELECTRICAL,5.93,-11.05,51.73
INSURANCE,4.45,0.29,34.85
PHARMA,3.87,-1.34,33.77
AUTO,3.34,-20.90,72.75


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3052856.0,21
AR,1299435.0,10
XR,1294620.0,13
X40,1015719.0,14
X40N,752708.0,9
OX40N,697014.0,10
XY25,349426.0,6
SR,281837.0,2
MH,74128.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3580550.0,25
M-SC,1349554.0,15
X-MC,1224981.0,16
X-LC,918302.0,11
X-SC,734477.0,8
H-MC,407963.0,3
L-SC,260430.0,3
H-LC,132968.0,2
M-LC,120039.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1220819.0      6
           AR         882367.0      5
           XR         796498.0      7
M-SC       XY24       790598.0      6
X-MC       X40        446414.0      7
X-LC       X40        395962.0      5
X-MC       XY24       346792.0      3
H-SC       OX40N      324901.0      4
X-SC       X40N       290154.0      3
M-SC       OX40N      285574.0      5
H-SC       SR         281837.0      2
X-SC       XY24       270980.0      3
X-MC       X40N       248454.0      4
X-LC       XY24       233256.0      2
H-MC       AR         217552.0      2
X-LC       X40N       214100.0      2
H-MC       XY24       190411.0      1
X-MC       XY25       183321.0      2
L-SC       XR         173891.0      2
X-SC       X40        173343.0      2
M-SC       XR         149246.0      2
           AR         124136.0      2
M-LC       XR         120039.0      1
L-SC       OX40N       86539.0      1
H-LC       AR          75380.0      1
X-LC       XY25        74984.0      2
H-SC       MH          74128.0      1
H-LC       X200        57588.0      1
L-MC       XR          54946.0      1
M-MC       XY25        49575.0      1
L-LC       XY25        41546.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
